# Ćwiczenie 3

Celem ćwiczenia jest imlementacja metody [Minimax z obcinaniem alpha-beta](https://en.wikipedia.org/wiki/Alpha%E2%80%93beta_pruning) do gry Connect Four (czwórki).

W trakcie ćwiczenia można skorzystać z reposytorium z implementacją gry [Connect Four udostępnionym przez Jakuba Łyskawę](https://github.com/lychanl/two-player-games). Ewentualnie, można zaimplementować samemu grę Connect Four (ale, tak aby rozwiązanie miało ten sam interfejs co podany poniżej).

Implementację Minimax należy przetestować używając różną głębokość przeszukiwania. Implementacja Solvera musi zapewniać interfejs jak poniżej, ale można dodać dowolne metody prywatne oraz klasy wspomagające (jeżeli będą potrzebne).

Punktacja:
- Działająca metoda Minimax - **2 pkt**
- Działająca metoda Minimax z obcinaniem alpha-beta - **1.5 pkt**
- Analiza jakości solvera w zależności od głębokości przeszukiwania **1.5pkt**
    - można zaimplementować w tym celu wizualizację rozgrywki dwóch agentów, bądź kilka przykładów 'z ręki'
- Jakość kodu **2pkt**

Aby importowanie elementów z poniższej komórki działało należy umieścić tego notebooka w tym samym folderze co paczkę `two_player_games`:
```
├── LICENSE
├── README.md
├── minimax.ipynb # HERE
├── test
│   ├── __init__.py
│   ├── test_connect_four.py
│   ├── test_dots_and_boxes.py
│   └── test_pick.py
└── two_player_games
    ├── __init__.py
    ├── games
    │   ├── connect_four.py
    │   └── dots_and_boxes.py
    ├── move.py
    ├── player.py
    └── state.py
```

In [2]:
from typing import Tuple, List
from copy import deepcopy
import math
from random import choice, shuffle
from two_player_games.player import Player
from two_player_games.games.connect_four import ConnectFour, ConnectFourMove

Wielkość planszy

In [3]:
ROW_COUNT = 6
COLUMN_COUNT = 7

evaluationTable = [[3, 4, 5, 7, 5, 4, 3], 
                    [4, 6, 8, 10, 8, 6, 4],
                    [5, 8, 11, 13, 11, 8, 5], 
                    [5, 8, 11, 13, 11, 8, 5],
                    [4, 6, 8, 10, 8, 6, 4],
                    [3, 4, 5, 7, 5, 4, 3]]

In [4]:
class MinMaxSolver:

    def __init__(self, game: ConnectFour):
        self.game = game

    def evaluate_position(self, state)->float:
        check_win = state.get_winner()
        if check_win:
            if check_win.char != state.get_current_player().char:
                return 1
            return -1
        return 0
    
    def evaluate_position_1(self, state)->float:
        player = state.get_current_player()
        board = state.state.fields
        score = 0
        utility = 138
        for ii in range(COLUMN_COUNT):
            for kk in range(ROW_COUNT):
                if board[ii][kk]:
                    if board[ii][kk].char == player.char:
                        score += evaluationTable[kk][ii]
                    else:
                        score -= evaluationTable[kk][ii]
        return (utility + score)


    def minimax(self, depth, state, is_maximizing_player:bool)-> Tuple[int, float]:
        """Returns column index and score"""
        if depth == 0 or state.is_finished():
            return (None, self.evaluate_position(state))
        
        if is_maximizing_player:
            pos_moves = self._get_valid_locations(state)
            value = -math.inf
            column = choice(pos_moves)
            for move in pos_moves:
                next_state = deepcopy(state)
                next_state.make_move(ConnectFourMove(move))
                new_value = self.minimax(depth-1, next_state, False)[1]
                if new_value > value:
                    value = new_value
                    column = move
        else:
            pos_moves = self._get_valid_locations(state)
            value = math.inf
            column = choice(pos_moves)
            for move in pos_moves:
                pos_moves = self._get_valid_locations(state)
                next_state = deepcopy(state)
                next_state.make_move(ConnectFourMove(move))
                new_value = self.minimax(depth-1, next_state, True)[1]
                if new_value < value:
                    value = new_value
                    column = move
        return (column, value)    
    
    def minimax_alfa_beta(self, depth, state, alpha, beta, is_maximizing_player:bool):

        if state.is_finished() or depth == 0:
            return (None, self.evaluate_position(state))
        
        if is_maximizing_player:
            pos_moves = self._get_valid_locations(state)
            shuffle(pos_moves)
            value = -math.inf
            column = choice(pos_moves)
            for move in pos_moves:
                pos_moves = self._get_valid_locations(state)
                next_state = deepcopy(state)
                next_state.make_move(ConnectFourMove(move))
                new_value = self.minimax_alfa_beta(depth-1, next_state, alpha, beta, False)[1]
                if new_value > value:
                    value = new_value
                    column = move
                alpha = max(alpha, value)
                if alpha >= beta:
                    break
        else:
            pos_moves = self._get_valid_locations(state)
            shuffle(pos_moves)
            value = math.inf
            column = choice(pos_moves)
            for move in pos_moves:
                next_state = deepcopy(state)
                next_state.make_move(ConnectFourMove(move))
                new_value = self.minimax_alfa_beta(depth-1, next_state, alpha, beta, True)[1]
                if new_value < value:
                    value = new_value
                    column = move
                beta = min(beta, value)
                if alpha >= beta:
                    break
        return (column, value)

    def _get_valid_locations(self, state)->List[int]:
        return [move.column for move in state.get_moves()] # dziala

    def _is_valid_move(self, state,  col_index:int)->bool:
        if state[col_index[-1]] is None:
            return True
        return False

Rozgrywka

In [9]:
p1 = Player("a")
p2 = Player("b")
game = ConnectFour(size=(COLUMN_COUNT, ROW_COUNT), first_player=p1, second_player=p2)
solver = MinMaxSolver(game)
is_max = False

'''game.make_move(ConnectFourMove(0))
game.make_move(ConnectFourMove(1))
game.make_move(ConnectFourMove(2))
game.make_move(ConnectFourMove(5))
game.make_move(ConnectFourMove(6))
game.make_move(ConnectFourMove(4))
game.make_move(ConnectFourMove(1))
game.make_move(ConnectFourMove(0))
game.make_move(ConnectFourMove(0))
game.make_move(ConnectFourMove(2))
game.make_move(ConnectFourMove(1))
game.make_move(ConnectFourMove(6))
game.make_move(ConnectFourMove(4))
game.make_move(ConnectFourMove(3))'''

#game.make_move(ConnectFourMove(3))
#game.make_move(ConnectFourMove(4))
#game.make_move(ConnectFourMove(3))
#game.make_move(ConnectFourMove(4))
#game.make_move(ConnectFourMove(3))
#game.make_move(ConnectFourMove(0))

'''game.make_move(ConnectFourMove(3))
game.make_move(ConnectFourMove(4))
game.make_move(ConnectFourMove(3))
game.make_move(ConnectFourMove(5))'''


solver = MinMaxSolver(game)
'''print(game)
next_move = solver.minimax_alfa_beta(5, game, -200, 200, True)
print(next_move)
if next_move[0] is not None:
    game.make_move(ConnectFourMove(next_move[0]))
print(game)'''

while(not game.is_finished()):
    solver.minimax_alfa_beta(5, game, -300, 300, True)
    next_move = solver.minimax_alfa_beta(5, game, -200, 200, True)
    if next_move[0] is not None:
        game.make_move(ConnectFourMove(next_move[0]))
    next_move = solver.minimax_alfa_beta(3, game, -200, 200, True)
    if next_move[0] is not None:
        game.make_move(ConnectFourMove(next_move[0]))
print(game)

Current player: b
[b][ ][b][ ][ ][ ][ ]
[a][ ][a][ ][ ][ ][a]
[b][a][b][b][ ][b][b]
[a][a][a][b][ ][a][a]
[b][a][b][a][ ][b][a]
[a][b][b][a][ ][a][b]
